In [1]:
import os
import pickle
import tqdm
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model


In [ ]:
tf.config.list_physical_devices('GPU')

In [ ]:
!nvidia-smi

In [ ]:
processed_clips_directory = '/scratch/tzhori.i/data_16'

file_names = [f for f in os.listdir(processed_clips_directory) if f.endswith('.pkl')]
all_clips = []

for file_name in tqdm.tqdm(file_names):
    try:
        processed_clip_path = os.path.join(processed_clips_directory, file_name)

        with open(processed_clip_path, 'rb') as f:
            clip_data = pickle.load(f)
            all_clips.append(clip_data)
    except:
        pass

In [ ]:
train_list, test_list = train_test_split(np.concatenate(all_clips, axis=0), test_size=0.1, random_state=0, shuffle=True)
print(f'train_shape: {train_list.shape} \ntest_shape: {test_list.shape}')

In [ ]:
def DSTCAE_C3D_256(win_length=16, img_width=128, img_height=128):
    temp_pool = 2
    temp_depth = 5

    # Model initialization
    model = models.Sequential()

    # Encoding
    model.add(layers.Conv3D(16, (5, 3, 3), activation='relu', padding='same', input_shape=(win_length, img_width, img_height, 1)))
    model.add(layers.MaxPooling3D((1, 2, 2), padding='same'))

    model.add(layers.Conv3D(32, (5, 3, 3), activation='relu', padding='same'))
    model.add(layers.MaxPooling3D((temp_pool, 2, 2), padding='same'))

    model.add(layers.Conv3D(64, (5, 3, 3), activation='relu', padding='same'))
    model.add(layers.MaxPooling3D((temp_pool, 2, 2), padding='same'))

    model.add(layers.Dropout(0.25))

    model.add(layers.Conv3D(128, (5, 3, 3), activation='relu', padding='same'))
    model.add(layers.MaxPooling3D((temp_pool, 2, 2), padding='same'))

    # Decoding
    model.add(layers.Conv3D(128, (5, 3, 3), activation='relu', padding='same'))
    model.add(layers.UpSampling3D((temp_pool, 2, 2)))

    model.add(layers.Conv3D(64, (5, 3, 3), activation='relu', padding='same'))
    model.add(layers.UpSampling3D((temp_pool, 2, 2)))

    model.add(layers.Conv3D(32, (5, 3, 3), activation='relu', padding='same'))
    model.add(layers.UpSampling3D((temp_pool, 2, 2)))

    model.add(layers.Conv3D(16, (5, 3, 3), activation='relu', padding='same'))
    model.add(layers.UpSampling3D((1, 2, 2)))
    
    model.add(layers.Conv3D(1, (5, 3, 3), activation='sigmoid', padding='same'))
    
    return model

autoencoder = DSTCAE_C3D_256()
autoencoder.summary()

In [ ]:
autoencoder.compile(optimizer='adam', loss='mean_squared_error')

In [ ]:
def Enhanced_DSTCAE_C3D_256(win_length=16, img_width=128, img_height=128):
    temp_pool = 2
    temp_depth = 5

    model = models.Sequential()

    # Encoding with more filters and additional layers
    model.add(layers.Conv3D(16, (7, 5, 5), activation='relu', padding='same', input_shape=(win_length, img_width, img_height, 1)))
    model.add(layers.MaxPooling3D((1, 2, 2), padding='same'))

    model.add(layers.Conv3D(32, (5, 3, 3), activation='relu', padding='same'))
    model.add(layers.MaxPooling3D((temp_pool, 2, 2), padding='same'))

    model.add(layers.Conv3D(64, (5, 3, 3), activation='relu', padding='same'))
    model.add(layers.MaxPooling3D((temp_pool, 2, 2), padding='same'))

    model.add(layers.Conv3D(128, (3, 3, 3), activation='relu', padding='same'))
    model.add(layers.MaxPooling3D((temp_pool, 2, 2), padding='same'))

    model.add(layers.Dropout(0.25))

    # Bottleneck
    model.add(layers.Conv3D(256, (3, 3, 3), activation='relu', padding='same'))

    # Decoding
    model.add(layers.Conv3D(128, (3, 3, 3), activation='relu', padding='same'))
    model.add(layers.UpSampling3D((temp_pool, 2, 2)))

    model.add(layers.Conv3D(64, (5, 3, 3), activation='relu', padding='same'))
    model.add(layers.UpSampling3D((temp_pool, 2, 2)))

    model.add(layers.Conv3D(31, (5, 3, 3), activation='relu', padding='same'))
    model.add(layers.UpSampling3D((temp_pool, 2, 2)))

    model.add(layers.Conv3D(16, (7, 5, 5), activation='relu', padding='same'))
    model.add(layers.UpSampling3D((1, 2, 2)))
    
    model.add(layers.Conv3D(1, (7, 5, 5), activation='sigmoid', padding='same'))
    
    return model

enhanced_autoencoder = Enhanced_DSTCAE_C3D_256()
enhanced_autoencoder.summary()

In [ ]:
enhanced_autoencoder.compile(optimizer='adam', loss='mean_squared_error')

In [ ]:
epochs = 10
batch_size = 8

enhanced_autoencoder.fit(train_list, train_list, epochs=epochs, batch_size=batch_size, shuffle=True)

In [ ]:
epochs = 10
batch_size = 4

simple_autoencoder.fit(train_list, train_list, epochs=epochs, batch_size=batch_size, shuffle=True)

In [ ]:
epochs = 5
batch_size = 8

autoencoder.fit(train_list, train_list, epochs=epochs, batch_size=batch_size, shuffle=True)

In [ ]:
autoencoder.save('128x128_10_8_all.h5')  # Creates a HDF5 file 'my_model.h5'

In [ ]:
model = load_model('256x256_5_8_all.h5')
model.compile(optimizer='adam', loss='mean_squared_error')

In [ ]:
epochs = 5
batch_size = 8
model.fit(train_list, train_list, epochs=epochs, batch_size=batch_size, shuffle=True)

In [ ]:
model.save('256x256_10_8_all.h5')  # Creates a HDF5 file 'my_model.h5'

In [ ]:
evaluation = autoencoder.evaluate(test_list, test_list)

In [ ]:
predicted = autoencoder.predict(test_list)

In [ ]:
def plot_reconstructed_images(original, reconstructed, n=3):
    plt.figure(figsize=(20, 8))
    for i in range(n):
        ax = plt.subplot(2, n, i + 1)
        plt.imshow(original[i].reshape(256, 256))
        plt.gray()
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)

        ax = plt.subplot(2, n, i + 1 + n)
        plt.imshow(reconstructed[i].reshape(256, 256))
        plt.gray()
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)
    plt.show()
    
n = 50
plot_reconstructed_images(test_list[n], predicted[n])